In [79]:
import pandas as pd
import numpy as np
import requests
import openmeteopy
from sklearn import preprocessing

#df = pd.read_csv('_ALL_FLIGHTS_30m.csv')
#df = pd.read_csv('flights_clean.csv')
df = pd.read_csv('USE_THESE.csv')
#df = pd.read_csv('flights_sample_3m.csv')

df2 = pd.read_csv('airports-code@public.csv', delimiter = ";")
df2.head()

,Airport Code,Airport Name,City Name,Country Name,Country Code,Latitude,Longitude,World Area Code,City Name geo_name_id,Country Name geo_name_id,coordinates
0,YCB,Cambridge Bay,Cambridge Bay,Canada,CA,69.116667,-105.133333,906,\N,6251999.0,"69.116667, -105.133333"
1,YKD,Township Airport,Kincardine,Canada,CA,44.201389,-81.606667,906,\N,6251999.0,"44.201389, -81.606667"
2,UPF,Pforheim,Pforheim,Germany,DE,48.900000,8.700000,429,\N,2921044.0,"48.9, 8.7"
3,IXB,Bagdogra,Bagdogra,India,IN,26.733333,88.333333,733,1277897,1269750.0,"26.733333000000002, 88.333333"
4,IMK,Simikot,Simikot,Nepal,NP,29.966667,81.833333,755,\N,1282988.0,"29.966667, 81.833333"


In [66]:
import asyncio

from open_meteo import OpenMeteo
from open_meteo.models import DailyParameters, HourlyParameters

In [67]:
import openmeteo_requests

import requests_cache
import pandas as pd

from openmeteo_sdk.Variable import Variable
from openmeteo_sdk.Aggregation import Aggregation
from retry_requests import retry

def getWeather(lat, lon, start_date, end_date):
    print(lat," lat       ", lon, " long")
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "rain", "snowfall",
                        "pressure_msl", "surface_pressure", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m",
                        "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m"]  
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    #print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
    #print(f"Elevation {response.Elevation()} m asl")
    #print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    #print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    hourly = response.Hourly()
    #hourly_temperature_2m = hourly.Variables(4).ValuesAsNumpy()

    # Process hourly data. The order of variables needs to be the same as requested.
    #hourly = response.Hourly()
    #hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

    #hourly_data = {"date": pd.date_range(
    #	start = pd.to_datetime(hourly.Time(), unit = "s"),
    #	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
    #	freq = pd.Timedelta(seconds = hourly.Interval()),
    #	inclusive = "left"
    #)}

    #hourly_data["temperature_2m"] = hourly_temperature_2m

    #hourly_dataframe = pd.DataFrame(data = hourly_data)

    #print(hourly_temperature_2m)
    #print(hourly_dataframe)
    #print(responses[0])
    return hourly
    
def addToDb(hourly, hour):
    
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_rain = hourly.Variables(4).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(5).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(6).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(7).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(8).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(9).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(10).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(11).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(12).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(13).ValuesAsNumpy()
    hourly_data = {}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data_hour = {}
    hourly_data_hour["FL_DATE"] = df['FL_DATE'][idx]
    hourly_data_hour["DOT_CODE"] = df['DOT_CODE'][idx]
    hourly_data_hour["FL_NUMBER"] = df['FL_NUMBER'][idx]
    hourly_data_hour["ORIGIN"] = df['ORIGIN'][idx]
    hourly_data_hour["temperature_2m"] = hourly_data["temperature_2m"][hour]
    hourly_data_hour["relative_humidity_2m"] = hourly_data["relative_humidity_2m"][hour]
    hourly_data_hour["dew_point_2m"] = hourly_data["dew_point_2m"][hour]
    hourly_data_hour["apparent_temperature"] = hourly_data["apparent_temperature"][hour]
    hourly_data_hour["rain"] = hourly_data["rain"][hour]
    hourly_data_hour["snowfall"] = hourly_data["snowfall"][hour]
    hourly_data_hour["pressure_msl"] = hourly_data["pressure_msl"][hour]
    hourly_data_hour["surface_pressure"] = hourly_data["surface_pressure"][hour]
    hourly_data_hour["vapour_pressure_deficit"] = hourly_data["vapour_pressure_deficit"][hour]
    hourly_data_hour["wind_speed_10m"] = hourly_data["wind_speed_10m"][hour]
    hourly_data_hour["wind_speed_100m"] = hourly_data["wind_speed_100m"][hour]
    hourly_data_hour["wind_direction_10m"] = hourly_data["wind_direction_10m"][hour]
    hourly_data_hour["wind_direction_100m"] = hourly_data["wind_direction_100m"][hour]
    hourly_data_hour["wind_gusts_10m"] = hourly_data["wind_gusts_10m"][hour]
    return hourly_data_hour
    
x = getWeather(52.52, 13.41, "2024-01-01", "2024-01-01")
#print(x)
hour = 4
print(addToDb(x, hour - 2))

52.52  lat        13.41  long
{'FL_DATE': '6/2/2023', 'DOT_CODE': 20363, 'FL_NUMBER': 4655, 'ORIGIN': 'JFK', 'temperature_2m': 4.6584997, 'relative_humidity_2m': 85.64379, 'dew_point_2m': 2.4585001, 'apparent_temperature': 0.8358574, 'rain': 0.0, 'snowfall': 0.0, 'pressure_msl': 1004.7, 'surface_pressure': 1000.01825, 'vapour_pressure_deficit': 0.12259996, 'wind_speed_10m': 14.458382, 'wind_speed_100m': 29.462873, 'wind_direction_10m': 198.88618, 'wind_direction_100m': 206.87807, 'wind_gusts_10m': 24.48}


In [13]:
df_meteo = pd.DataFrame(addToDb(x, hour - 1), index=[0])

#ONLY USE IF MERGING
#df_test = pd.read_csv('flights_clean.csv')
#df_meteo = pd.DataFrame(addToDb(x, hour - 1), index=[0])
#result_df = pd.concat([df, df_meteo], ignore_index=True)
#result_df.head()

import pandas as pd


# Assume you have two key columns
key_column1 = 'FL_DATE'
key_column2 = 'DOT_CODE'
key_column3 = 'FL_NUMBER'

# Merge DataFrames using two keys
merged_df = pd.merge(df, df_meteo, on=[key_column1, key_column2, key_column3])

# Display the merged DataFrame
print(merged_df)
#merged_df.to_csv("mergeme.csv", index=False)

NameError: name 'idx' is not defined

In [68]:
column_info = pd.DataFrame({
    'Column Name': df.columns,
    'Data Type': df.dtypes
})

print(column_info)

                         Column Name Data Type
FL_DATE                      FL_DATE    object
AIRLINE                      AIRLINE    object
AIRLINE_DOT              AIRLINE_DOT    object
AIRLINE_CODE            AIRLINE_CODE    object
DOT_CODE                    DOT_CODE     int64
FL_NUMBER                  FL_NUMBER     int64
ORIGIN                        ORIGIN    object
ORIGIN_CITY              ORIGIN_CITY    object
DEST                            DEST    object
DEST_CITY                  DEST_CITY    object
CRS_DEP_TIME            CRS_DEP_TIME     int64
ARR_TIME                    ARR_TIME   float64
CANCELLED                  CANCELLED     int64
ELAPSED_TIME            ELAPSED_TIME   float64
DISTANCE                    DISTANCE     int64
DELAY_DUE_WEATHER  DELAY_DUE_WEATHER   float64


In [69]:
 ##########################
# Functions for Point data #
 ##########################
import re

minYearIndex = 0
maxYearIndex = 4

minMonthIndex = 5
maxMonthIndex = 7

minDayIndex = 8
maxDayIndex = 11

def getHourMinuteinApiFormat(depTime):
    depTime1 = 0
    #if len(depTime) == 2:
        #print(depTime)
        #time = depTime.zfill(4)
        #h = time[0:2]
        #min = time[2:4]
    if len(depTime) <= 3: #if the hour is before 10 basically
        depTime1 = depTime.zfill(4)
        #print(depTime1[0:2])
        return depTime1[0:2], depTime1[2:4]

    h = depTime[0:2]
    min = depTime[2:4]
    #print("depTime ", depTime)
    #print("h ", h)
    return h, min


#def getDateinApiFormat(date):

    #d = date[minDayIndex:maxDayIndex] 
    #m = date[minMonthIndex:maxMonthIndex] 
    #y = date[minYearIndex:maxYearIndex] 
    #return d, m, y

def getWeatherInfo(date):

    return -1

def getTimeStampFromDateTime64(dateTime):
    dateTime = data.index
    return Timestamp(np.datetime64(dateTime[0]))

def getLatLongofAirport(airportCode):
    codeRow = df2.loc[df2['Airport Code'] == airportCode]
    if codeRow.size == 0:
        print("ERROR with ", airportCode) #add these airports manually in dataset until you dont get this log running the driver
        return "-999","-999"
    codeRowValues = codeRow["coordinates"].values[0]
    lat = codeRowValues.split(',', 1)[0]  
    long = codeRowValues.split(',', 1)[1]  
    return lat, long

#print(getHourMinuteinApiFormat(str(row["CRS_DEP_TIME"])))
#date = "2023-04-13"
#print(getDateinApiFormat(date))

In [58]:
print(df["FL_DATE"])

KeyError: 'FL_DATE'

In [80]:
from datetime import datetime
import csv

key_column1 = 'FL_DATE'
key_column2 = 'DOT_CODE'
key_column3 = 'FL_NUMBER'
key_column4 = 'ORIGIN'
whatever = pd.DataFrame({'FL_DATE': [], 'AIRLINE': [], 'AIRLINE_DOT': [],
                          'AIRLINE_CODE': [], 'DOT_CODE': [], 'FL_NUMBER': [], 'ORIGIN': [],
                          'ORIGIN_CITY': [], 'DEST': [], 'DEST_CITY': [], 'CRS_DEP_TIME': [],
                          'ARR_TIME': [], 'CANCELLED': [], 'ELAPSED_TIME': [], 'DISTANCE': [],
                          'DELAY_DUE_WEATHER': [], 'temperature_2m': [], 'relative_humidity_2m': [],
                          'dew_point_2m': [], 'apparent_temperature': [], 'rain': [], 'snowfall': [],
                          'pressure_msl': [], 'surface_pressure': [], 'vapour_pressure_deficit': [],
                          'wind_speed_10m': [], 'wind_speed_100m': [], 'wind_direction_10m': [],
                          'wind_direction_100m': [], 'wind_gusts_10m': []})
merged_df = []
acc = 0
#dataframe addition drviver function, this one iterates the dataframe, returns it with added weather info column(s) like dew point, precip, etc.
for idx, row in df.iterrows():
    acc = acc + 1
    lat, long = getLatLongofAirport(row["ORIGIN"])
    if lat != "-999":
        #print("REC ", lat)
        #d, m, y = getDateinApiFormat(row["FL_DATE"])
 
        h, min = getHourMinuteinApiFormat(str(row["CRS_DEP_TIME"]))
        
        date_str = row["FL_DATE"]
        date_object = datetime.strptime(date_str, '%m/%d/%Y')
        formatted_date = date_object.strftime('%Y-%m-%d')
        w = getWeather(float(lat), float(long), formatted_date, formatted_date)
    
        df_meteo = pd.DataFrame(addToDb(w, int(h) - 2), index=[idx])
        #new_data = {'new_column1': 42, 'new_column2': 'some_value'}  # Add your new columns and values
        #new_columns_df = pd.DataFrame(new_data, index=[0])

        #row_index = 0  # Replace with the index of the row you want to modify
        
        #NEED 4 KEYS

        # Merge DataFrames using three keys
        
        xxx = (pd.merge(df, df_meteo, on=[key_column1, key_column2, key_column3, key_column4])).values.flatten().tolist()
        merged_df.append(xxx)
        #print(merged_df)
        #whatever.append(merged_df)
        #whatever = pd.merge(df, df_meteo, on=[key_column1, key_column2, key_column3, key_column4])
        print("The current index is:", idx)
        if acc >= 7500:
            break
        #print(whatever.head())
        #newdf = pd.DataFrame(data, index=[0])
        #whatever.to_csv("probsnot.csv", index=False)
        
        #start = datetime(int(y), int(m), int(d), int(h), 0)
        #end = datetime(int(y), int(m), int(d), int(h)-1, 0)
        #print(start)
        #print(end)
        #print(location)
        #print(weather)
        #getWeatherData(start, end, location, h, m)
       # start = datetime(y, m, d, h, 0)
       # end = datetime(y, m, d, h-1, 0)
       # vancouver = Point(49.2497, -123.1193, 70)

        # print("Day: ", d)
        # print("Month: ", m)
        # print("Year: ", y)
        #print(row)
        #print(h)
new_df = pd.DataFrame(merged_df, columns = whatever.columns)
#merged_df = pd.concat(new_df)
new_df.to_csv("data64.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)
print(idx)

38.851944  lat        -77.037778  long
The current index is: 0
40.6925  lat        -74.168611  long
The current index is: 1
34.895556  lat        -82.218889  long
The current index is: 2
38.944444  lat        -77.455833  long
The current index is: 3
40.193611  lat        -76.763333  long
The current index is: 4
41.938889  lat        -72.683333  long
The current index is: 5
40.6925  lat        -74.168611  long
The current index is: 6
40.6925  lat        -74.168611  long
The current index is: 7
42.940556  lat        -78.732222  long
The current index is: 8
40.6925  lat        -74.168611  long
The current index is: 9
32.133333  lat        -81.2  long
The current index is: 10
40.6925  lat        -74.168611  long
The current index is: 11
40.6925  lat        -74.168611  long
The current index is: 12
43.118889  lat        -77.6725  long
The current index is: 13
40.6925  lat        -74.168611  long
The current index is: 14
40.6925  lat        -74.168611  long
The current index is: 15
40.6925  

29.984444  lat        -95.341389  long
The current index is: 150
33.433333  lat        -112.033333  long
The current index is: 151
38.851944  lat        -77.037778  long
The current index is: 152
43.116667  lat        -76.116667  long
The current index is: 153
35.216667  lat        -80.933333  long
The current index is: 154
38.851944  lat        -77.037778  long
The current index is: 155
39.871944  lat        -75.241111  long
The current index is: 156
42.95  lat        -87.9  long
The current index is: 157
39.871944  lat        -75.241111  long
The current index is: 158
42.733333  lat        -73.8  long
The current index is: 159
42.2125  lat        -83.353333  long
The current index is: 160
39.291667  lat        -94.730556  long
The current index is: 161
42.940556  lat        -78.732222  long
The current index is: 162
38.851944  lat        -77.037778  long
The current index is: 163
38.851944  lat        -77.037778  long
The current index is: 164
42.2125  lat        -83.353333  long
The

The current index is: 289
27.533333  lat        -99.466667  long
The current index is: 290
30.083333  lat        -94.1  long
The current index is: 291
32.896944  lat        -97.038056  long
The current index is: 292
32.896944  lat        -97.038056  long
The current index is: 293
31.616667  lat        -97.233333  long
The current index is: 294
35.616667  lat        -106.083333  long
The current index is: 295
32.896944  lat        -97.038056  long
The current index is: 296
35.333333  lat        -94.366667  long
The current index is: 297
35.616667  lat        -106.083333  long
The current index is: 298
32.896944  lat        -97.038056  long
The current index is: 299
32.896944  lat        -97.038056  long
The current index is: 300
32.896944  lat        -97.038056  long
The current index is: 301
39.223056  lat        -106.868889  long
The current index is: 302
33.9425  lat        -118.407222  long
The current index is: 303
33.433333  lat        -112.033333  long
The current index is: 304
3

The current index is: 423
39.861667  lat        -104.673056  long
The current index is: 424
35.393056  lat        -97.600833  long
The current index is: 425
38.036389  lat        -84.605833  long
The current index is: 426
40.850833  lat        -96.759167  long
The current index is: 427
36.198333  lat        -95.888056  long
The current index is: 428
47.123611  lat        -88.573333  long
The current index is: 429
42.883333  lat        -85.533333  long
The current index is: 430
41.978611  lat        -87.904722  long
The current index is: 431
39.499167  lat        -119.768056  long
The current index is: 432
41.978611  lat        -87.904722  long
The current index is: 433
33.9425  lat        -118.407222  long
The current index is: 434
31.9425  lat        -102.201944  long
The current index is: 435
30.194444  lat        -97.67  long
The current index is: 436
39.861667  lat        -104.673056  long
The current index is: 437
30.194444  lat        -97.67  long
The current index is: 438
37.618

The current index is: 552
39.861667  lat        -104.673056  long
The current index is: 553
39.861667  lat        -104.673056  long
The current index is: 554
39.861667  lat        -104.673056  long
The current index is: 555
39.861667  lat        -104.673056  long
The current index is: 556
37.618889  lat        -122.375  long
The current index is: 557
39.861667  lat        -104.673056  long
The current index is: 558
37.151944  lat        -94.498333  long
The current index is: 559
33.9425  lat        -118.407222  long
The current index is: 560
33.9425  lat        -118.407222  long
The current index is: 561
39.861667  lat        -104.673056  long
The current index is: 562
41.866667  lat        -103.6  long
The current index is: 563
39.861667  lat        -104.673056  long
The current index is: 564
39.861667  lat        -104.673056  long
The current index is: 565
40.983333  lat        -124.1  long
The current index is: 566
40.983333  lat        -124.1  long
The current index is: 567
39.2230

...

The current index is: 6859
32.733333  lat        -117.183333  long
The current index is: 6860
32.733333  lat        -117.183333  long
The current index is: 6861
32.733333  lat        -117.183333  long
The current index is: 6862
32.733333  lat        -117.183333  long
The current index is: 6863
32.733333  lat        -117.183333  long
The current index is: 6864
32.733333  lat        -117.183333  long
The current index is: 6865
32.733333  lat        -117.183333  long
The current index is: 6866
32.733333  lat        -117.183333  long
The current index is: 6867
32.733333  lat        -117.183333  long
The current index is: 6868
32.733333  lat        -117.183333  long
The current index is: 6869
32.733333  lat        -117.183333  long
The current index is: 6870
32.733333  lat        -117.183333  long
The current index is: 6871
32.733333  lat        -117.183333  long
The current index is: 6872
32.733333  lat        -117.183333  long
The current index is: 6873
32.733333  lat        -117.183333  

The current index is: 7012
38.748611  lat        -90.37  long
The current index is: 7013
38.748611  lat        -90.37  long
The current index is: 7014
38.748611  lat        -90.37  long
The current index is: 7015
38.748611  lat        -90.37  long
The current index is: 7016
38.748611  lat        -90.37  long
The current index is: 7017
27.975556  lat        -82.533056  long
The current index is: 7018
32.116111  lat        -110.941111  long
The current index is: 7019
32.116111  lat        -110.941111  long
The current index is: 7020
30.194444  lat        -97.67  long
The current index is: 7021
30.194444  lat        -97.67  long
The current index is: 7022
30.194444  lat        -97.67  long
The current index is: 7023
36.124444  lat        -86.678056  long
The current index is: 7024
34.2  lat        -118.35  long
The current index is: 7025
34.2  lat        -118.35  long
The current index is: 7026
34.2  lat        -118.35  long
The current index is: 7027
34.2  lat        -118.35  long
The cu

The current index is: 7157
39.175278  lat        -76.668333  long
The current index is: 7158
39.175278  lat        -76.668333  long
The current index is: 7159
39.175278  lat        -76.668333  long
The current index is: 7160
39.175278  lat        -76.668333  long
The current index is: 7161
39.175278  lat        -76.668333  long
The current index is: 7162
39.175278  lat        -76.668333  long
The current index is: 7163
39.175278  lat        -76.668333  long
The current index is: 7164
39.175278  lat        -76.668333  long
The current index is: 7165
39.175278  lat        -76.668333  long
The current index is: 7166
39.175278  lat        -76.668333  long
The current index is: 7167
39.175278  lat        -76.668333  long
The current index is: 7168
39.175278  lat        -76.668333  long
The current index is: 7169
39.175278  lat        -76.668333  long
The current index is: 7170
39.175278  lat        -76.668333  long
The current index is: 7171
39.175278  lat        -76.668333  long
The curren

The current index is: 7305
39.175278  lat        -76.668333  long
The current index is: 7306
32.898611  lat        -80.040556  long
The current index is: 7307
41.416667  lat        -81.85  long
The current index is: 7308
40.0  lat        -82.883333  long
The current index is: 7309
40.0  lat        -82.883333  long
The current index is: 7310
38.816667  lat        -104.716667  long
The current index is: 7311
32.852778  lat        -96.85  long
The current index is: 7312
38.851944  lat        -77.037778  long
The current index is: 7313
38.851944  lat        -77.037778  long
The current index is: 7314
39.861667  lat        -104.673056  long
The current index is: 7315
39.861667  lat        -104.673056  long
The current index is: 7316
39.861667  lat        -104.673056  long
The current index is: 7317
39.861667  lat        -104.673056  long
The current index is: 7318
39.861667  lat        -104.673056  long
The current index is: 7319
42.2125  lat        -83.353333  long
The current index is: 73

The current index is: 7454
32.733333  lat        -117.183333  long
The current index is: 7455
32.733333  lat        -117.183333  long
The current index is: 7456
32.733333  lat        -117.183333  long
The current index is: 7457
32.733333  lat        -117.183333  long
The current index is: 7458
32.733333  lat        -117.183333  long
The current index is: 7459
32.733333  lat        -117.183333  long
The current index is: 7460
32.733333  lat        -117.183333  long
The current index is: 7461
32.733333  lat        -117.183333  long
The current index is: 7462
32.733333  lat        -117.183333  long
The current index is: 7463
32.733333  lat        -117.183333  long
The current index is: 7464
32.733333  lat        -117.183333  long
The current index is: 7465
32.733333  lat        -117.183333  long
The current index is: 7466
32.733333  lat        -117.183333  long
The current index is: 7467
32.733333  lat        -117.183333  long
The current index is: 7468
32.733333  lat        -117.183333  

In [17]:
from datetime import datetime
from meteostat import Hourly
from pandas import Timestamp
import numpy as np

# Set time period
start = datetime(2018, 1, 1, 22, 59)
end = datetime(2018, 1, 1, 23, 1)
vancouver = Point(49.2497, -123.1193, 70)

print(type(start))
print(type(end))
print(type(vancouver))

# Get hourly data
#data = Hourly(vancouver, start, end)
#data = data.fetch()
#print(data)

#print(getTimeStampFromDateTime64(data.index[0]))
#getLatLongofAirport("")
#print(data.get_loc)


ModuleNotFoundError: No module named 'meteostat'